### Import

In [1]:
import dateutil.parser
import glob,os

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster import KMeansClusterer

import numpy as np
import pandas as pd

from gensim.models import Doc2Vec, Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Loading Data

In [2]:
path = 'C:\\Users\\ab356917\\Downloads\\Cluster-data'
data = []
raw_data = []
for infile in glob.glob( os.path.join(path, '*.*') ):
       with open(infile) as file:
            string = file.readlines()
            raw_data.extend(string)
            for each_line in string:
                data.extend(each_line.split(":")[2:])

In [3]:
tokenized_sentences = sent_tokenize(str(data))
# tokenized_words = []
# for sent in data:
#     tokenized_words.append(word_tokenize(sent))

In [4]:
# max_ = 0
# for i in tokenized_words:
#     #print(len(i))
#     if len(i)>max_:
#         max_ = len(i)

# new_tokenized_words = [] 
# for words in tokenized_words:
#     if len(words)<max_:
#         number_of_words_to_add = max_ - len(words)
#         words_to_add = ['ZERO']*number_of_words_to_add
#         words.extend(words_to_add)
#     new_tokenized_words.append(words)

# Word2Vec

In [5]:
# # train model
# model = Word2Vec(new_tokenized_words, min_count=1)
# X = model[list(model.wv.vocab)]
# NUM_CLUSTERS=3
# kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
# assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
# print (assigned_clusters)

In [6]:
# words = list(model.wv.vocab)
# for i, word in enumerate(words):
#     if assigned_clusters[i] == 1:
#         print (word + ":" + str(assigned_clusters[i]))

# DOC2VEC

In [7]:
sentenceLabeled = []
for sentenceID, sentence in enumerate(data):
    sentenceL = TaggedDocument(words=sentence.split(), tags = ['SENT_%s' %sentenceID])
    sentenceLabeled.append(sentenceL)

In [16]:
sentenceLabeled

[TaggedDocument(words=['www04-005.logmeinrescue-enterprise.com'], tags=['SENT_0']),
 TaggedDocument(words=['443'], tags=['SENT_1']),
 TaggedDocument(words=['Welcome', 'to', 'HP', 'Customer', 'Support', 'for', 'Notebook', 'Support.', 'My', 'name', 'is', 'Akash', 'and', 'I', 'will', 'be', 'your', 'technical', 'support', 'specialist', 'today.'], tags=['SENT_2']),
 TaggedDocument(words=['Hi!', 'How', 'are', 'you', 'doing', 'today', '?'], tags=['SENT_3']),
 TaggedDocument(words=['Not', 'good', 'my', 'new', 'computer', 'is', 'not', 'working'], tags=['SENT_4']),
 TaggedDocument(words=['Do', 'not', 'worry,', 'I', 'will', 'do', 'my', 'best', 'to', 'help', 'you./'], tags=['SENT_5']),
 TaggedDocument(words=['If', 'I', 'am', 'correct,', 'after', 'logging', 'in', 'the', 'icons', 'take', 'time', 'to', 'appear.'], tags=['SENT_6']),
 TaggedDocument(words=['My', 'computer', 'wont', 'boot', 'up', 'and', 'it', 'goes', 'to', 'the', 'home', 'page', 'and', 'lets', 'me', 'put', 'in', 'my', 'password', 'but',

In [8]:
doc2vec_model = Doc2Vec(size=300, window=10, min_count=0, workers=11, alpha=0.005, min_alpha=0.025)
doc2vec_model.build_vocab(sentenceLabeled)

C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [9]:
for epoch in range(20):
    print(epoch ,"is running")
    doc2vec_model.train(sentenceLabeled,total_examples=len(tokenized_sentences),epochs=20000)
    doc2vec_model.alpha -= 0.0008  # decrease the learning rate
    doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
    doc2vec_model.save("doc2vec_model-withoutTime.model")
textVect = doc2vec_model.docvecs.doctag_syn0

0 is running
1 is running
2 is running
3 is running
4 is running
5 is running
6 is running
7 is running
8 is running
9 is running
10 is running
11 is running
12 is running
13 is running
14 is running
15 is running
16 is running
17 is running
18 is running
19 is running


C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  import sys


In [10]:
## K-means ##
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(textVect)
clusters = km.labels_.tolist()

In [11]:
## Print Sentence Clusters ##
cluster_info = {'sentence': data, 'cluster' : clusters}
sentenceDF = pd.DataFrame(cluster_info, index=[clusters], columns = ['sentence','cluster'])

In [12]:
for num in range(num_clusters):
    print()
    print("Sentence cluster %d: " %int(num+1), end='')
    print()
    for sentence in sentenceDF.ix[num]['sentence'].values.tolist():
        print(' %s ' %sentence, end='')
        print()
    print()


Sentence cluster 1: 
  www04-005.logmeinrescue-enterprise.com 
 443
 
  Hi! How are you doing today ?
 
  Alright.
 
  Alright.
 
  Alright=.
 
  Please cancel the test.
 
  Canelled it
 
  Restarting it
 
  Alright=.
 
  Alright.
 
  Please enter the password/
 
  Entered it
 
  Now please type "msconfig".
 
  Yes.
 
  Ok in service's 
 
  Great.
 
  control.app04-07.logmeinrescue-enterprise.com (64.74.103.155 
 443)
 
  Hi, how are you doing today?
 
  ***********
 
  Thank you for sharing the phone number.
 
  yes, same system
 
 //www.hp.com/go/privacy.
 
  Thank you for the permission.
 
  ok
 
  control.app04-07.logmeinrescue-enterprise.com (64.74.103.155 
 443)
 
  control.app05-05.logmeinrescue-enterprise.com (63.251.34.137 
 443)
 
  Welcome back.
 
  Great.
 
  Let me explain.
 
  ok
 
  control.app04-07.logmeinrescue-enterprise.com (64.74.103.155 
 443)
 
  Welcome back.
 
 //www.support.hp.com.
 
  control.app05-04.logmeinrescue-enterprise.com (63.251.34.136 
 443)
 
  Hi


C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


# cluster 1 -other && c2-ans c3-ques

In [ ]:
# from nltk.cluster.kmeans import KMeansClusterer
# kclusterer = KMeansClusterer(3, distance=nltk.cluster.util.cosine_distance, repeats=25)
# assigned_clusters = kclusterer.cluster(textVect, assign_clusters=True)
# ## Print Sentence Clusters ##
# cluster_info1 = {'sentence': data, 'cluster' : assigned_clusters}
# sentenceDF1 = pd.DataFrame(cluster_info1, index=[clusters], columns = ['sentence','cluster'])
# for num in range(3):
#     print()
#     print("Sentence cluster %d: " %int(num+1), end='')
#     print()
#     for sentence in sentenceDF1.ix[num]['sentence'].values.tolist():
#         print(' %s ' %sentence, end='')
#         print()
#     print()

# HDBSCAN

In [13]:
golden_set_vectors = [doc2vec_model.infer_vector(x.words, alpha=0.05, min_alpha=0.001, steps=50) for x in sentenceLabeled]
X = np.asarray(golden_set_vectors)
distances = pairwise_distances(X, metric='cosine').astype('float64')

from hdbscan import HDBSCAN

clusterer = HDBSCAN(cluster_selection_method='leaf', algorithm='best', approx_min_span_tree=True, gen_min_span_tree=True, leaf_size=40, metric="precomputed",min_cluster_size=3, min_samples=None, p=None, core_dist_n_jobs=-1)

clusterer.fit(distances)
set(clusterer.labels_)

{-1, 0, 1, 2, 3, 4, 5, 6}

In [14]:
p = clusterer.labels_

In [15]:
for i, word in enumerate(data): 
    if p[i] == 1:
        print (word + ":" + str(p[i]))

 Homepage password now
:1
 Let me explain.
:1
 Is it asking for code
:1
 yes, working now thank you
:1
 good bye and thank you again:1
 Is this working fine now?
:1
